## Import All Packages That I Need

In [ ]:
from Diarization.AdditionFunctions import create_labelling, SpectralClusterer, EigenGapType 
from Diarization.audio import Dpreprocess_wav
from Diarization.voice_encoder import VoiceEncoder

from ASR.AdditionFunctions import load_asr_model, predict_text

from pathlib import Path
from tqdm import tqdm
import json
import librosa

In [ ]:
# Define your paths using pathlib

audio_folder_path = Path('./Data/DataForDiarization/')
modelD_file_path = Path('./Models/DIARIZATIONMODEL.pt')
modelA_file_path = Path('./Models/ASRMODEL.weights.h5')
modelA_Config_file_path = Path('./Models/ASRMODELCONFIG.json')
diarization_folder_path = Path('./Output/DiarizationResults')


## Load Dirization Model

In [ ]:
# Load the voice encoder model
encoder = VoiceEncoder("cpu", weights_fpath=modelD_file_path)

# Configure the spectral clusterer
clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=10,
    eigengap_type=EigenGapType.Ratio
)

In [ ]:
# Process each audio file in the folder
audio_files = Path(audio_folder_path).glob('*.wav')
for audio_file_path in tqdm(audio_files, desc="Processing audio files"):
    wav = Dpreprocess_wav(audio_file_path)
    _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)

    labels = clusterer.predict(cont_embeds)
    predicted_data = create_labelling(labels, wav_splits)

    # Save the labelling to a JSON file with the same name as the audio file
    output_file_path = Path(diarization_folder_path) / f"{audio_file_path.stem}.json"
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(predicted_data, f, ensure_ascii=False, indent=4)
        
print("Processing completed.")

## Load ASR Model

In [ ]:
ASR_model = load_asr_model(modelA_Config_file_path, modelA_file_path)

In [ ]:
# Process each diarization file in the folder
diarization_files = Path(diarization_folder_path).glob('*.json')
for diarization_path in tqdm(diarization_files, desc="Processing diarization files"):
    # Load the diarization segments
    with open(diarization_path, 'r', encoding='utf-8') as file:
        diarization_segments = json.load(file)

    # Load the corresponding audio file
    audio_file_name = diarization_path.stem + '.wav'
    audio_path = Path(audio_folder_path) / audio_file_name
    audio, sr = librosa.load(audio_path, sr=None)

    # Predict text for each diarization segment with a progress bar
    for segment in tqdm(diarization_segments, desc="Processing segments"):
        start_sample = int(segment['start'] * sr)
        end_sample = int(segment['end'] * sr)
        segment_audio = audio[start_sample:end_sample]
        text = predict_text(segment_audio, ASR_model)
        segment['text'] = text

    # Overwrite the existing JSON file with the new predictions
    with open(diarization_path, 'w', encoding='utf-8') as file:
        json.dump(diarization_segments, file, ensure_ascii=False, indent=4)

print("Processing completed.")

# Thanks ALL